In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
df = pd.read_excel('../input/amazon-seller-order-status-prediction/orders_data.xlsx')


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.simplefilter('ignore')
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords

%matplotlib inline

In [ ]:
!pip install openpyxl

In [7]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head(3)

In [8]:
# Convert city and state into uppercase
places = ['ship_city', 'ship_state']
for i in places:
    df[i]=df[i].apply(lambda x:x.upper())

In [9]:
# removing comma from city values
df['ship_city']=df['ship_city'].apply(lambda x:x.replace(',', ''))
df['ship_state'] = df['ship_state'].apply(lambda x: x.replace(',', ''))

In [ ]:
df.isnull().sum()

In [10]:
df['shipping_fee'].fillna(df['shipping_fee'].mode()[0], inplace=True)
df['item_total'].fillna(df['item_total'].mode()[0], inplace=True)
df['cod'].fillna('online', inplace=True)

In [11]:
# remove rupee symbol from amount features
amounts = ['item_total', 'shipping_fee']
for i in amounts:
    df[i] = df[i].apply(lambda x: x.replace(',', ''))        # remove the commas in amounts greater than 1000
    df[i] = df[i].apply(lambda x: x[1:])

In [13]:
# change data types
i = 'int64'
f = 'float64'
df = df.astype({'item_total': f, 'shipping_fee': f, 'quantity': i})

In [ ]:
df.head(3)

In [14]:
df[[ 'cod', 'order_status', 'item_total']].groupby(['cod', 'order_status']).sum().reset_index()

In [ ]:
fig = px.histogram(df, x="cod", y="item_total", color='order_status', barmode='group', height=400)
fig.show()

In [15]:
df['year']=pd.DatetimeIndex(df['order_date']).year

In [17]:
df['date'] = pd.DatetimeIndex(df['order_date']).date
df['time'] = pd.DatetimeIndex(df['order_date']).time
df['month_name'] = pd.DatetimeIndex(df['date']).month_name()
df['day_name'] = pd.DatetimeIndex(df['date']).day_name()

In [18]:
df=df.drop( columns=['order_date'])


In [ ]:
df.head(2)

In [19]:
df['sku'] = df['sku'].apply(lambda x: x[4:])
df.head(2)

In [20]:
df_sales=df[['date', 'item_total']].groupby('date').sum().reset_index()


In [21]:
df_sales.head(3)

In [22]:
fig = px.line(df_sales, x='date', y="item_total", title = 'Sales over the period')
fig.show()

In [24]:
df_days = df[['year', 'month_name', 'day_name', 'item_total']].groupby(['year', 'month_name', 'day_name']).sum().reset_index()
# df_days.sort_values(by=['year', 'month_name'])

In [26]:
fig = px.bar(df_days, x="month_name", y="item_total", color="day_name", title="Month-wise Sales")
fig.show()

In [27]:
df_places = df[['ship_state', 'ship_city', 'item_total']].groupby(['ship_state', 'ship_city']).sum().reset_index()
df_places.sort_values(by='item_total', ascending=False, inplace=True)

In [28]:
fig = px.sunburst(data_frame=df_places, path=['ship_state', 'ship_city'], values='item_total', title='Region-wise orders')
fig.show()

In [29]:
df.groupby('ship_city').size()

In [30]:
top_cities = df.groupby('ship_city').size().reset_index().rename(columns={0: 'Total'}).sort_values('Total', ascending=False).head()
fig = px.pie(top_cities, values='Total', names='ship_city', color_discrete_sequence=px.colors.sequential.RdBu, title='Top 5 ordering cities')
fig.show()

In [31]:
top_cities

In [32]:
top_cities = df.groupby('sku').size().reset_index().rename(columns={0: 'Total'}).sort_values('Total', ascending=False).head()
fig = px.pie(top_cities, values='Total', names='sku', color_discrete_sequence=px.colors.sequential.BuGn_r, title='Top 5 Products')
fig.show()

In [33]:
df['order_status'].value_counts()

In [34]:
df_status = df[['order_status', 'item_total']]
fig = px.bar(data_frame=df_status, x='order_status', y='item_total', color='order_status', title='Success rate of orders')
fig.show()

In [35]:
df['description'] = df['description'].apply(lambda x: x.replace('|', ''))

In [36]:
stop_words = stopwords.words('english')
stop_words.extend(['Pure', 'Leather', 'Shantiniketan', 'with', 'clean_dedsc', 'phone', 'for', 'and', 'print', 'block', 'dtype'])

In [37]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stop_words:
            result.append(token)
            
    return ' '.join(result)

In [38]:
# cleaned data
df['clean_desc'] = df['description'].apply(preprocess)

In [39]:
plt.figure(figsize=(15,10))
wc = WordCloud(max_words=2000, width=1600, height=800, stopwords=stop_words).generate(str(df[df['order_status']=='Delivered to buyer'].clean_desc))
plt.title('Most Popular words', fontsize=20)
plt.imshow(wc)